In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.analyze as analyze
import thor.option as option
import thor.attribute as attribute
import thor.utils as utils

notebook_name = "cpol_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [7]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T18:00:00"
end = "2005-11-13T20:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
output_directory.mkdir(parents=True, exist_ok=True)
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

# if output_directory.exists():
#     shutil.rmtree(output_directory)

times_dict = {"start": start, "end": end}
cpol_options = data.option.CPOLOptions(**times_dict)
era5_dict = {"latitude_range": [-14, -10], "longitude_range": [129, 133]}
era5_pl_options = data.option.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.option.ERA5Options(**times_dict, **era5_dict)
data_options = data.option.DataOptions(
    datasets=[cpol_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")

altitude = list(np.arange(0, 20e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="cpol")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-11-01 21:06:01,125 - thor.data.option - INFO - Generating cpol filepaths.
2024-11-01 21:06:01,128 - thor.data.option - INFO - Assigning default era5 pressure levels.
2024-11-01 21:06:01,129 - thor.data.option - INFO - Assigning default era5 pressure-levels options name and fields.
2024-11-01 21:06:01,130 - thor.data.option - INFO - Generating era5 filepaths.
2024-11-01 21:06:01,133 - thor.data.option - INFO - Assigning default era5 single-levels options name and fields.
2024-11-01 21:06:01,136 - thor.data.option - INFO - Generating era5 filepaths.


2024-11-01 21:06:01,152 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-11-01 21:06:01,153 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.


In [8]:
times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-11-01 21:06:03,317 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-11-01 21:06:03,321 - thor.track - INFO - Beginning simultaneous tracking.
2024-11-01 21:06:03,442 - thor.track - INFO - Processing 2005-11-13T18:00:08.
2024-11-01 21:06:03,443 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T18:00:08.
2024-11-01 21:06:03,444 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-11-01 21:06:03,484 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-11-01 21:06:05,916 - thor.track - INFO - Processing hierarchy level 0.
2024-11-01 21:06:05,918 - thor.track - INFO - Tracking convective.
2024-11-01 21:06:15,419 - thor.track - INFO - Tracking middle.
2024-11-01 21:06:15,422 - thor.track - INFO - Tracking anvil.
2024-11-01 21:06:15,425 - thor.track - INFO - Processing hierarchy level 1.
2024-11-01 21:06:15,426 - thor.track - INFO 

# Cartesian Coordinates

In [11]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_directory.exists():
    shutil.rmtree(output_directory)

grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data_options.to_yaml(options_directory / "data.yml")
track_options.to_yaml(options_directory / "track.yml")
)
# visualize.option.save_display_options(visualize_options, options_directory)

times = data.utils.generate_times(data_options.dataset_by_name("cpol"))
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

SyntaxError: unmatched ')' (3935311195.py, line 13)

# Analysis

In [8]:
analysis_options = analyze.mcs.analysis_options()
utils.save_options(analysis_options, filename="analysis", options_directory=output_directory / "options")
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory)

,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 18:00:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 18:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:20:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:30:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:40:09,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 18:50:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:00:08,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 19:10:23,1,trailing,left,left,up-shear,up-shear
2005-11-13 19:20:09,1,trailing,left,leading,up-shear,up-shear


In [16]:
figure_options = visualize.option.horizontal_attribute_options(
    "cpol_20051113", style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64("2005-11-13T14:00")
end_time = np.datetime64("2005-11-13T20:00")
visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options, parallel_figure=True, by_date=False)

2024-10-31 13:11:56,671 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:00:08.000000000.
2024-10-31 13:11:56,674 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.180000.nc
2024-10-31 13:11:58,174 - thor.visualize.attribute - INFO - Saving cpol_20051113 figure for 2005-11-13T18:00:08.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather 

2024-10-31 13:12:32,204 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:10:23.000000000.
2024-10-31 13:12:32,218 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.181000.nc
2024-10-31 13:12:32,494 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:20:09.000000000.
2024-10-31 13:12:32,513 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.182000.nc
2024-10-31 13:12:33,455 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:30:09.000000000.
2024-10-31 13:12:33,464 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.183000.nc
2024-10-31 13:12:33,574 - thor.visualize.attribute - INFO - Visualizing MCS at time 2005-11-13T18:40:09.000000000.
2024-10-31 13:12:33,588 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.184000.nc
2024-10-31 13:12:33,908 - thor.visualize.attribute - INFO - Saving cpol_20051113

KeyboardInterrupt: 